# Import Libraries

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import pandas as pd
from timecopilot import TimeCopilot


# load environment variables

TimeCopilot uses [Pydantic](https://ai.pydantic.dev/) to interact with LLMs, which has explicit support for [many common model providers](https://ai.pydantic.dev/models/overview/) and implicit support for endpoints using  OpenAI compatible endpoints.

Configuration will vary slightly depending on which provider is being used. For example, when using OpenAI as your model provider the `OPENAI_API_KEY` environment variable will need to be set to your api key.

### directly set the variable with python

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your-new-secret-key"

### Load from the shell

This needs to be done before starting your Python process.

In [ ]:
# bash
export OPENAI_API_KEY="your-new-secret-key"

In [ ]:
# powershell
setx OPENAI_API_KEY "your-new-secret-key"

### store in a .env file and load with [python-dotenv](https://pypi.org/project/python-dotenv/)

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

# LLM Providers

For a full list of LLM providers officially supported by TimeCopilot via Pydantic go to [Pydantic's list of model providers](https://ai.pydantic.dev/models/overview/). Each of those model providers has its own setup section in Pydantic's documentation.

## OpenAI

### Set the environment variable

`OPENAI_API_KEY` is the required environment variable for OpenAI. Load it with your preferred method, 3 methods are described above.

In [ ]:
# load the .env file with python-dotenv
from dotenv import load_dotenv

load_dotenv()

### Initialize the forecasting agent


In [4]:
tc = TimeCopilot(
    llm="openai:gpt-4o",
    retries=3
)

## Ollama

### Set the environment variables

With Ollama the `OLLAMA_BASE_URL` environment variable is required and the `OLLAMA_API_KEY` environment variable may be needed depending on your ollama configuration. If you are running your model locally, you probably don't need the api key.

You could also use Ollama Cloud, which requires both environment variables with `OLLAMA_BASE_URL` being set to `https://ollama.com/v1`

The default url when running ollama locally is `http://localhost:11434/v1`.

Load your environment variables with your preferred method, 3 methods are described above. 

In [ ]:
# load the .env file with python-dotenv
from dotenv import load_dotenv

load_dotenv()

### Initialize the forecasting agent

#### Simple approach (use OllamaProvider by name)

In [ ]:
tc = TimeCopilot(
    llm='ollama:gpt-oss:20b',
    retries=3
)

#### Initialize the model and provider directly

In [ ]:
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.ollama import OllamaProvider

llm = OpenAIChatModel(
    model_name="gpt-oss:20b",
    provider=OllamaProvider(base_url="http://localhost:11434/v1"),
)

tc = TimeCopilot(
    llm=llm,
    retries=3
)

## implicitly supported model providers

In some cases, there are providers that are compatible with Pydantic that are not on the list of officially supported providers. Generally these will be providers that have an OpenAI style endpoint, but no provider included in Pydantic for simple string initialization.

In cases like this, you'll initialize the model and provider directly with the appropriate arguments for your setup.



In [ ]:
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.openai import OpenAIProvider

LLM_API_BASE_URL = "http://127.0.0.1:1234/v1"
MODEL_NAME = "gpt-oss-20b"
API_KEY = "api-key"

model = OpenAIChatModel(
    MODEL_NAME,
    provider=OpenAIProvider(
        base_url=LLM_API_BASE_URL,
        api_key=API_KEY,
    ),
)

tc = TimeCopilot(
    llm=model,
    retries=3,
)

# Use TimeCopilot

After setting up your chosen model and initializing TimeCopilot, you can use it as usual.

In [12]:
df = pd.read_csv("https://timecopilot.s3.amazonaws.com/public/data/air_passengers.csv")
print(df.head())

       unique_id          ds    y
0  AirPassengers  1949-01-01  112
1  AirPassengers  1949-02-01  118
2  AirPassengers  1949-03-01  132
3  AirPassengers  1949-04-01  129
4  AirPassengers  1949-05-01  121


In [6]:
result = tc.forecast(df=df, freq="MS")

1it [00:00,  7.48it/s]
1it [00:00, 25.92it/s]
1it [00:00, 251.31it/s]
0it [00:00, ?it/s]15:51:06 - cmdstanpy - INFO - Chain [1] start processing
15:51:06 - cmdstanpy - INFO - Chain [1] done processing
1it [00:03,  3.55s/it]
15:51:15 - cmdstanpy - INFO - Chain [1] start processing
15:51:15 - cmdstanpy - INFO - Chain [1] done processing
0it [00:00, ?it/s]15:51:18 - cmdstanpy - INFO - Chain [1] start processing
15:51:19 - cmdstanpy - INFO - Chain [1] done processing
1it [00:02,  2.97s/it]15:51:21 - cmdstanpy - INFO - Chain [1] start processing
15:51:21 - cmdstanpy - INFO - Chain [1] done processing
2it [00:05,  2.97s/it]15:51:24 - cmdstanpy - INFO - Chain [1] start processing
15:51:25 - cmdstanpy - INFO - Chain [1] done processing
3it [00:09,  3.03s/it]15:51:28 - cmdstanpy - INFO - Chain [1] start processing
15:51:28 - cmdstanpy - INFO - Chain [1] done processing
4it [00:12,  3.05s/it]15:51:31 - cmdstanpy - INFO - Chain [1] start processing
15:51:31 - cmdstanpy - INFO - Chain [1] done pro

In [7]:
print(result.output.tsfeatures_analysis)

The time series analysis reveals several key attributes which will guide our model selection:

1. **Stability (0.933)**: A higher value indicates a relatively stable time series, which is consistent with the seasonal pattern typical in a dataset like AirPassengers.
2. **Unit Root Tests**: The PP test (-6.57) suggests the series is stationary, whereas the KPSS test (2.74) indicates non-stationarity. This mixed signal is somewhat expected in real-world data and suggests models that handle both trends and seasonality might perform well.
3. **Trend and Seasonality**: The series includes a strong trend (0.997) and pronounced seasonal strength (0.982). This points to models that can effectively capture both components, such as Prophet, which explicitly models both trend and seasonality.
4. **Series Length (144)**: A reasonably long series enables the usage of sophisticated models that require more data to perform effectively.


In [8]:
print(result.fcst_df)

        unique_id         ds     Prophet
0   AirPassengers 1961-01-01  466.560401
1   AirPassengers 1961-02-01  461.042082
2   AirPassengers 1961-03-01  493.413542
3   AirPassengers 1961-04-01  492.113653
4   AirPassengers 1961-05-01  496.445709
5   AirPassengers 1961-06-01  537.592041
6   AirPassengers 1961-07-01  577.166093
7   AirPassengers 1961-08-01  577.599117
8   AirPassengers 1961-09-01  529.038266
9   AirPassengers 1961-10-01  493.889181
10  AirPassengers 1961-11-01  460.030234
11  AirPassengers 1961-12-01  489.392785
12  AirPassengers 1962-01-01  502.415939
13  AirPassengers 1962-02-01  496.321423
14  AirPassengers 1962-03-01  531.969966
15  AirPassengers 1962-04-01  528.065107
16  AirPassengers 1962-05-01  534.174659
17  AirPassengers 1962-06-01  573.615281
18  AirPassengers 1962-07-01  614.245102
19  AirPassengers 1962-08-01  614.206790
20  AirPassengers 1962-09-01  566.306418
21  AirPassengers 1962-10-01  530.606803
22  AirPassengers 1962-11-01  497.766797
23  AirPassenger

In [10]:
query_result = tc.query("how many passengers will be in total in the next months?")
print(query_result.output)

To calculate the total number of passengers forecasted for the next months using the Prophet model, I will sum up the forecasted values. Here are the forecasted passenger numbers for the next several months:

1. 466.56
2. 461.04
3. 493.41
4. 492.11
5. 496.45
6. 537.59
7. 577.17
8. 577.60
9. 529.04
10. 493.89
11. 460.03
12. 489.39

Adding these values gives us a total of approximately \( 6,074.98 \) passengers forecasted over the next 12 months.

This forecast is based on the Prophet model, which has a MASE score of 1.09, indicating its estimation accuracy is relatively good compared to other models.


# Model Compatibility

TimeCopilot is not compatible with every model, tool use is required a required feature for a model to be compatible with TimeCopilot.